encoder transformer层的linear1层（etl1）替换为近似矩阵乘法

In [25]:
import numpy as np
import os
import matmul as mm
import math_util as mu
import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [26]:
# root_path = '/data/hdr/csi_transformer/pic/'
# print(root_path)
# root_path2 = '/'.join(root_path.split("/")[:-2])
# # sys.path.append(root_path)
# print(root_path2)

In [27]:
# method = METHOD_MITHRAL
# method = METHOD_PQ
method = METHOD_MITHRALPQ
# method = METHOD_EXACT
# method = METHOD_SCALAR_QUANTIZE
nbits = 8 # 量化比特数

In [28]:
linear_name = 'etl2'
feedback_bits = 256
ncodebooks = 64 # max:512
ncentroids = 64
if method == METHOD_MITHRAL:
    ncentroids = 16
train_sam_num = 3000 # 训练集样本数
test_sam_num = 1000 # 测试集样本数(如需修改，请同时修改下面的读取文件，现文件默认1000个样本)
batch_size = 32
if method == METHOD_EXACT:
    ncodebooks = 0
    ncentroids = 0

In [29]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
    dir_result = 'F:\\Projects\\python\\PQ\\res'
    data_to_fcpath_train= ''
    featurepath_train= ''
    data_to_fcpath_test = ''
    featurepath_test = ''
elif host_name == 'jm-System-Product-Name':
    dir_train = os.path.join('/data/hdr/transformer_data/joined', 'train', 'f'+str(feedback_bits))
    dir_test = os.path.join('/data/hdr/transformer_data/joined', 'test', 'f'+str(feedback_bits))
    dir_result = '/data/hdr/pq/res'
    data_to_fcpath_train= 'ex_linear2in_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    y_train = 'ex_linear2_y_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    featurepath_train= 'ex_linear2out_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    data_to_fcpath_test = 'ex_linear2in_test.npy'
    featurepath_test = 'ex_linear2out_test.npy'
    # y_test = 'y_test.npy'
else:
    raise NameError("You are running the script in a new computer, please define dir_train")


weightpath = 'ex_linear2_w_f256.npy'
biaspath = 'ex_linear2_b_f256.npy'
dir_result = os.path.join(dir_result, method, "f%i" % feedback_bits, linear_name)
try:
    os.mkdir(dir_result)
except FileNotFoundError:
    os.makedirs(dir_result)
except FileExistsError:
    pass 


In [30]:
est3 = mm.estFactory(X_path=data_to_fcpath_train, W_path=weightpath, Y_path=y_train, dir= dir_train, ncodebooks=ncodebooks, ncentroids=ncentroids, methods=[method], nbits=nbits)


running method:  PQ
running kmeans in subspace 1/256... mse / {var(X_subs), var(X)}: 0.256, 0.00461
running kmeans in subspace 2/256... mse / {var(X_subs), var(X)}: 0.314, 0.00602
running kmeans in subspace 3/256... mse / {var(X_subs), var(X)}: 0.013, 0.00262
running kmeans in subspace 4/256... mse / {var(X_subs), var(X)}: 3.38e-05, 0.000401
running kmeans in subspace 5/256... mse / {var(X_subs), var(X)}: 0.199, 0.0183
running kmeans in subspace 6/256... mse / {var(X_subs), var(X)}: 0.14, 0.0385
running kmeans in subspace 7/256... mse / {var(X_subs), var(X)}: 0.6, 0.0345
running kmeans in subspace 8/256... mse / {var(X_subs), var(X)}: 0.0901, 0.00143
running kmeans in subspace 9/256... X.shape:  (51200, 2)
k:  256
nnz_rows:  63


ValueError: could not broadcast input array from shape (63,2) into shape (2,)

: 

In [ ]:
x_test = np.load(dir_test+'/'+data_to_fcpath_test)
w_test = np.load(dir_train+'/'+weightpath)
bias = np.load(dir_train+'/'+biaspath)
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [ ]:
print(y_out_last)
print("y_out_last.shape: ", y_out_last.shape)
y_out_last_re = y_out_last.reshape(test_sam_num, batch_size, -1, y_out_last.shape[-1])
print("y_out_last_re.shape: ", y_out_last_re.shape)
if method == METHOD_EXACT:
    train_sam_num = 0 # 训练集样本数
if method == METHOD_SCALAR_QUANTIZE:
    np.save(os.path.join(dir_result, '%s%s_trsam%i_tesam%i_fb%i_nbits%i.npy' % (method, linear_name, train_sam_num, test_sam_num, feedback_bits, nbits)), y_out_last_re.astype(np.float32))
else:
    np.save(os.path.join(dir_result, '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i.npy' % (method, linear_name, train_sam_num, test_sam_num, feedback_bits, ncodebooks, ncentroids)), y_out_last_re)

[[ 0.50646554  0.10646586  0.51386934 ...  0.22492261  0.02745226
   0.47220983]
 [ 0.46864788  0.12992632  0.47988059 ...  0.25832577  0.03483174
   0.42371113]
 [ 0.50793373  0.10451188  0.50128958 ...  0.22104703  0.01779238
   0.46261187]
 ...
 [ 0.43996474  0.16640982  0.20484331 ...  0.22249337  0.19770805
   0.24413645]
 [ 0.30084064 -0.06226786  0.10596441 ...  0.29139958  0.07577209
   0.30812661]
 [ 0.31353394  0.12219785  0.18652727 ...  0.01956101  0.13025346
   0.19029885]]
y_out_last.shape:  (1024000, 64)
y_out_last_re.shape:  (1000, 32, 32, 64)
